# Class 1. Hello Quantum World (Exercises)

EVA: Quantum Machine Learning | ZHAW | Pavel Sulimov

Goals of this practice session:

1. Set up the quantum computing environment (Qiskit, PennyLane).
2. Work with qubits, superposition, and the Bloch sphere.
3. Run first quantum circuits and interpret measurement results.

## Prerequisites

```
pip install "qiskit>=2.0,<3" "qiskit-aer>=0.15" "pennylane>=0.40" matplotlib numpy
```

Tested with Python 3.11, Qiskit 2.x, PennyLane 0.40+.

---
## Part 1: Math tasks

### M1.1. Probabilities and normalization

Given $|\psi\rangle = \frac{1}{\sqrt{3}}|0\rangle + \sqrt{\frac{2}{3}}\,e^{i\pi/4}|1\rangle$:

1. Compute $P(0)$ and $P(1)$.
2. Verify that $P(0) + P(1) = 1$ (normalization).
3. Does the phase factor $e^{i\pi/4}$ affect the probabilities?

In [ ]:
import numpy as np

alpha = ...  # YOUR CODE HERE: amplitude of |0>
beta = ...   # YOUR CODE HERE: amplitude of |1> (include the phase)

P_0 = ...  # YOUR CODE HERE
P_1 = ...  # YOUR CODE HERE

print(f"P(0) = {P_0}")
print(f"P(1) = {P_1}")
print(f"P(0) + P(1) = {P_0 + P_1}")
# Does the phase e^(i*pi/4) affect P(1)? Explain in a comment below.
# YOUR ANSWER HERE

### M1.2. Plus and minus states

Write $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$ and $|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$ in column vector form.

Verify that they are:
1. Normalized: $\langle +|+\rangle = 1$, $\langle -|-\rangle = 1$.
2. Orthogonal: $\langle +|-\rangle = 0$.

In [ ]:
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

ket_plus = ...   # YOUR CODE HERE
ket_minus = ...  # YOUR CODE HERE

print(f"|+> = {ket_plus}")
print(f"|-> = {ket_minus}")

# Verify normalization and orthogonality using np.dot and .conj()
# YOUR CODE HERE

### M1.3. Bloch sphere coordinates

A general single-qubit state can be parameterized as $|\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\varphi}\sin(\theta/2)|1\rangle$, which maps to the Bloch vector $(\sin\theta\cos\varphi,\; \sin\theta\sin\varphi,\; \cos\theta)$.

Identify $\theta$ and $\varphi$ for $|0\rangle$, $|1\rangle$, $|+\rangle$, and $|i\rangle = \frac{|0\rangle + i|1\rangle}{\sqrt{2}}$.

In [ ]:
# Fill in the theta and phi values for each state.
states = {
    "|0>": {"theta": ..., "phi": ...},   # YOUR CODE HERE
    "|1>": {"theta": ..., "phi": ...},   # YOUR CODE HERE
    "|+>": {"theta": ..., "phi": ...},   # YOUR CODE HERE
    "|i>": {"theta": ..., "phi": ...},   # YOUR CODE HERE
}

print("State     theta      phi        Bloch (x, y, z)")
print("-" * 55)
for name, data in states.items():
    theta, phi = data["theta"], data["phi"]
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    print(f"{name:5s}  {theta:8.4f}  {phi:8.4f}   ({x:5.2f}, {y:5.2f}, {z:5.2f})")

---
## Part 2: Programming tasks

### P1.1. First Qiskit circuit

In the lecture we saw the Hadamard gate applied to $|0\rangle$, yielding a 50/50 split. Here, build a circuit that creates an *unequal* superposition using a rotation gate $R_y(\pi/3)$.

By default, every qubit in a quantum circuit starts in the state $|0\rangle$. To prepare a different initial state, you would apply gates: for example, `qc.x(0)` flips the qubit to $|1\rangle$ before any other operation.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram

qc = QuantumCircuit(1)
# YOUR CODE HERE: apply Ry(pi/3) to qubit 0, then add measurements

print(qc.draw("text"))

In [ ]:
sampler = StatevectorSampler()
# YOUR CODE HERE: run the circuit with 1024 shots, extract and print counts
# Then call plot_histogram(counts)

### P1.2. The same rotation in PennyLane

Implement $R_y(\pi/3)|0\rangle$ in PennyLane and compare the result with Qiskit. PennyLane's `default.qubit` device returns exact probabilities by default (no shot noise), so you can verify the analytical answer directly.

In [ ]:
import pennylane as qml
import matplotlib.pyplot as plt

dev = qml.device("default.qubit", wires=1)

theta = np.pi / 3


@qml.qnode(dev)
def ry_circuit():
    # YOUR CODE HERE: apply RY(theta) and return probabilities
    ...


probs = ry_circuit()
print(f"P(|0>) = {probs[0]:.4f},  P(|1>) = {probs[1]:.4f}")
print(f"Analytical: P(|0>) = {np.cos(theta/2)**2:.4f},  "
      f"P(|1>) = {np.sin(theta/2)**2:.4f}")

### P1.2b. PennyLane with shot noise

On real quantum hardware, you do not get exact probabilities. Instead, you sample individual outcomes. Specifying `shots` in PennyLane simulates this finite-sampling noise.

In [ ]:
dev_shots = qml.device("default.qubit", wires=1, shots=1024)


@qml.qnode(dev_shots)
def ry_circuit_shots():
    # YOUR CODE HERE: same circuit, but return qml.counts()
    ...


counts_pl = ry_circuit_shots()
print(f"Counts (1024 shots): {counts_pl}")

### P1.3. Bloch sphere visualization

In the lecture demo, we visualized the six cardinal states. Here, repeat the exercise with states created by rotation gates at various angles, to build intuition for how gate parameters move the state on the Bloch sphere.

We plot six states: three produced by $R_y(\theta)|0\rangle$ at different angles and three produced by $R_z(\varphi)|+\rangle$ at different phases.

In [ ]:
from qiskit.quantum_info import Statevector


def statevector_to_bloch(sv):
    """Convert a 2-element statevector to Bloch coordinates (x, y, z)."""
    a, b = sv[0], sv[1]
    bx = 2 * np.real(a * np.conj(b))
    by = 2 * np.imag(a * np.conj(b))
    bz = np.abs(a)**2 - np.abs(b)**2
    return [bx, by, bz]


# YOUR CODE HERE: define state_defs as a dictionary mapping state names
# to lists of (gate_name, parameter, qubit) tuples.
# Include Ry(pi/6), Ry(pi/3), Ry(2*pi/3) on |0>,
# and Rz(0), Rz(pi/2), Rz(pi) applied after H on |0>.
state_defs = {
    # YOUR CODE HERE
}

bloch_vectors = {}
for name, gates in state_defs.items():
    qc_tmp = QuantumCircuit(1)
    for gate_name, param, qubit in gates:
        if param is not None:
            getattr(qc_tmp, gate_name)(param, qubit)
        else:
            getattr(qc_tmp, gate_name)(qubit)

    sv = Statevector.from_instruction(qc_tmp)
    bloch_vectors[name] = statevector_to_bloch(sv.data)
    bx, by, bz = bloch_vectors[name]
    print(f"{name:14s}  Bloch=({bx:+.2f}, {by:+.2f}, {bz:+.2f})")

In [ ]:
# Plotting code (provided). Run after completing the cell above.
fig, axes = plt.subplots(
    2, 3, figsize=(15, 10), subplot_kw={"projection": "3d"}
)

u = np.linspace(0, 2 * np.pi, 30)
v = np.linspace(0, np.pi, 20)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

for idx, (name, vec) in enumerate(bloch_vectors.items()):
    ax = axes.flat[idx]
    ax.plot_wireframe(
        x_sphere, y_sphere, z_sphere, alpha=0.1, color="gray"
    )
    for sign in (-1.3, 1.3):
        ax.plot([sign, -sign], [0, 0], [0, 0], "k-", alpha=0.2)
        ax.plot([0, 0], [sign, -sign], [0, 0], "k-", alpha=0.2)
        ax.plot([0, 0], [0, 0], [sign, -sign], "k-", alpha=0.2)

    ax.quiver(
        0, 0, 0, vec[0], vec[1], vec[2],
        color="red", arrow_length_ratio=0.1, linewidth=3,
    )
    ax.scatter([vec[0]], [vec[1]], [vec[2]], color="red", s=100)
    ax.set_title(name, fontsize=14)
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_zlim(-1.3, 1.3)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

plt.suptitle(
    "Top row: Ry moves along a meridian (Y-Z plane). "
    "Bottom row: Rz rotates around the equator.",
    fontsize=13,
)
plt.tight_layout()
plt.show()

---
## Summary

Qubits are described by complex amplitudes $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ with $|\alpha|^2 + |\beta|^2 = 1$. Measurement yields probabilistic outcomes governed by the Born rule: $P(k) = |\langle k|\psi\rangle|^2$. The Bloch sphere provides a geometric picture of single-qubit states.

On the software side, Qiskit and PennyLane offer complementary interfaces to the same underlying linear algebra. Shot noise is an inherent feature of quantum measurement; the estimation error scales as $1/\sqrt{N}$.

Next session: quantum gates, multi-qubit systems, entanglement, and Bell states.